In [87]:
import pandas as pd
import itertools

from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error

In [105]:
test_df = pd.read_csv(r'C:\Repositorio\Housing Prices Competition\test.csv') #Test data to 

train_df = pd.read_csv(r'C:\Repositorio\Housing Prices Competition\train.csv') #Test data to 

results_df = pd.DataFrame()

imp = SimpleImputer(strategy = 'most_frequent')

headers = train_df.columns.values 

headers_test = test_df.columns.values 

train_df = imp.fit_transform(train_df) 

test_df = imp.fit_transform(test_df) 

train_df = pd.DataFrame(data = train_df, columns=headers)

test_df = pd.DataFrame(data = test_df, columns=headers_test)

error_results = []

house_features = []

features = []

for header in headers_test:
    if type(train_df[header][0]) is not str:
        
        features.append(header)

print(features)

for L in range(1, len(features)+1):
    for feature in itertools.combinations(features, L):    
        #Data to predict
        y = train_df.SalePrice
        #Features to check
        house_features = list(feature)
        print(house_features)
        #Data of from the selected features
        X = train_df[house_features]
        #Define the model 
        house_model = DecisionTreeRegressor( random_state=1 )
        # Split the data
        train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)
        #Fit model
        house_model.fit(train_X, train_y)

        #Check the model

        val_predictions = house_model.predict(X)
        iteration_error = mean_absolute_error(y, val_predictions)
        error_results.append(iteration_error)

        if (min(error_results) == iteration_error):

           Final_prediction = test_df[house_features]
           results_df = pd.DataFrame()
           results_df['Id'] = test_df['Id']
           results_df['SalePrice'] = house_model.predict(Final_prediction)

print(iteration_error)

results_df.to_csv(r'C:\Repositorio\Housing Prices Competition\Results.csv', index = False)




tFinSF1', 'FullBath', 'TotRmsAbvGrd', 'EnclosedPorch']
['MasVnrArea', 'BsmtFinSF1', 'FullBath', 'TotRmsAbvGrd', '3SsnPorch']
['MasVnrArea', 'BsmtFinSF1', 'FullBath', 'TotRmsAbvGrd', 'ScreenPorch']
['MasVnrArea', 'BsmtFinSF1', 'FullBath', 'TotRmsAbvGrd', 'PoolArea']
['MasVnrArea', 'BsmtFinSF1', 'FullBath', 'TotRmsAbvGrd', 'MiscVal']
['MasVnrArea', 'BsmtFinSF1', 'FullBath', 'TotRmsAbvGrd', 'MoSold']
['MasVnrArea', 'BsmtFinSF1', 'FullBath', 'TotRmsAbvGrd', 'YrSold']
['MasVnrArea', 'BsmtFinSF1', 'FullBath', 'Fireplaces', 'GarageYrBlt']
['MasVnrArea', 'BsmtFinSF1', 'FullBath', 'Fireplaces', 'GarageCars']
['MasVnrArea', 'BsmtFinSF1', 'FullBath', 'Fireplaces', 'GarageArea']
['MasVnrArea', 'BsmtFinSF1', 'FullBath', 'Fireplaces', 'WoodDeckSF']
['MasVnrArea', 'BsmtFinSF1', 'FullBath', 'Fireplaces', 'OpenPorchSF']
['MasVnrArea', 'BsmtFinSF1', 'FullBath', 'Fireplaces', 'EnclosedPorch']
['MasVnrArea', 'BsmtFinSF1', 'FullBath', 'Fireplaces', '3SsnPorch']
['MasVnrArea', 'BsmtFinSF1', 'FullBath', 'Fir

KeyboardInterrupt: 

In [ ]:
print(features)